**Table of contents**<a id='toc0_'></a>    
- 1. [Imports](#toc1_)    
- 2. [Solve DP](#toc2_)    
- 3. [Simulation](#toc3_)    
- 4. [Policy functions](#toc4_)    
- 5. [Save](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Imports](#toc0_)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from NonConvexDurablesModelEGM import NonConvexDurablesModelEGMClass


## 2. <a id='toc2_'></a>[Solve DP](#toc0_)

In [19]:
model_DP = NonConvexDurablesModelEGMClass()


In [20]:
model_DP.link_to_cpp()

In [ ]:
model_DP.solve()
model_DP.simulate_R()
model_DP.compute_transfer_func()
model_DP.sim.R

In [22]:
sim = model_DP.sim
par = model_DP.par
egm = model_DP.egm

## 3. <a id='toc3_'></a>[Simulation](#toc0_)

In [ ]:
print(f'{sim.R = :12.8f}')

In [ ]:
fig = plt.figure(figsize=(12,8))

ax = fig.add_subplot(2,2,1)
ax.plot(np.mean(model_DP.sim.c, axis=1))
ax.set_title('con')

ax = fig.add_subplot(2,2,2)
ax.plot(np.mean(model_DP.sim.d,axis=1))
ax.set_title('d')

ax = fig.add_subplot(2,2,3)
ax.plot(np.mean(model_DP.sim.DC,axis=1))
ax.set_title('adj')

ax = fig.add_subplot(2,2,3)
ax.plot(np.mean(model_DP.sim.states[...,0],axis=1))
ax.set_title('m')

fig.tight_layout()

## 4. <a id='toc4_'></a>[Policy functions](#toc0_)

In [ ]:

for t in [0,5,10,14]:

    fig = plt.figure(figsize=(6,4))
    
    ax = fig.add_subplot(1,1,1)
    for i_p in [egm.Np//5,2*egm.Np//5]:
        for i_n,ls in zip([egm.Nn//3,2*egm.Nn//3],['-','--']):
            y = (1-model_DP.egm.sol_sav_share_keep[t,i_p,i_n,:]) * model_DP.egm.m_grid
            ax.plot(egm.m_grid,y,ls=ls,label=f"p={egm.p_grid[i_p]:.2f}, n={egm.n_grid[i_n]:.2f}")
    
    ax.legend()

    ax.set_title(f't={t}')

In [ ]:
for t in [0,5,10,14]:

    fig,ax = plt.subplots(1,2)
    
    # ax = fig.a
    for i_p in [egm.Np//5,2*egm.Np//5,3*egm.Np//5]:
        for i_n,ls in zip([egm.Nn//3,2*egm.Nn//3],['-','--']):
            x =  egm.m_grid + (1-par.kappa) * egm.n_grid[i_n]
            expenditure = (1-egm.sol_exp_share_adj[t,i_p,i_n]) * x
            c = egm.sol_c_share_adj[t,i_p,i_n] * expenditure
            d = expenditure - c
            ax[0].plot(egm.m_grid,c,ls=ls,label=f"p={egm.p_grid[i_p]:.2f}, n={egm.n_grid[i_n]:.2f}")
            ax[1].plot(egm.m_grid,d,ls=ls,label=f"p={egm.p_grid[i_p]:.2f}, n={egm.n_grid[i_n]:.2f}")

    
    ax[0].legend()
    ax[0].set_title('c')
    ax[1].set_title('d')


    fig.suptitle(f't={t}')

## 5. <a id='toc5_'></a>[Save](#toc0_)

In [14]:
model_DP.save('../output/NonConvexDurablesModel_1D_DP_full.pkl')

In [15]:
# a. grids for dynamic states


# keeper
egm.sol_sav_share_keep = None
egm.sol_v_keep = None

# adjuster
egm.sol_exp_share_adj = None
egm.sol_c_share_adj = None
egm.sol_v_adj = None

egm.sol_func_evals_keep = None
egm.sol_flag_keep = None
egm.sol_func_evals_adj = None
egm.sol_flag_adj = None

# post-decision
egm.sol_w = None

In [17]:
model_DP.save('../output/NonConvexDurablesModel_1D_DP.pkl')